In [3]:
# ===============================
# Connect Four Simulation Notebook
# ===============================

import sys
import os
import numpy as np
from itertools import combinations

# --- Add SRC folder to path ---
src_path = os.path.join(os.getcwd(), "src")
if src_path not in sys.path:
    sys.path.append(src_path)

# --- Imports ---
from board import create_board, make_move, check_win
from player import random_player, heuristic_player, intelligent_player
from simulation import simulate_game  # full game simulation

# --- Parameters ---
PRE_SIM = 50   # small pre-simulation to estimate balance
BASE_SIM = 100 # minimum simulations
MAX_SIM = 500  # maximum simulations

# --- Players ---
players = {
    "4 Years": random_player,
    "7 Years": heuristic_player,
    "9 Years": lambda b, p: intelligent_player(b, p, depth=1),
    "11 Years": lambda b, p: intelligent_player(b, p, depth=2)
}

# --- Determine simulation count from balance ---
def choose_sim_count(prob_p1, prob_p2):
    """Choose number of simulations based on how balanced the players are."""
    balance = abs(prob_p1 - prob_p2)  # 0 = perfectly balanced, 1 = very unbalanced
    sim_count = int(np.clip(BASE_SIM + (1 - balance) * (MAX_SIM - BASE_SIM), BASE_SIM, MAX_SIM))
    return sim_count

# --- Run simulations with pre-sim ---
for p1_name, p2_name in combinations(players.keys(), 2):

    # --- Pre-Simulation to estimate balance ---
    pre_outcomes = {p1_name:0, p2_name:0, "Draw":0}
    for _ in range(PRE_SIM):
        winner = simulate_game(players[p1_name], players[p2_name])
        if winner == 1:
            pre_outcomes[p1_name] += 1
        elif winner == 2:
            pre_outcomes[p2_name] += 1
        else:
            pre_outcomes["Draw"] += 1

    prob_p1_pre = pre_outcomes[p1_name] / PRE_SIM
    prob_p2_pre = pre_outcomes[p2_name] / PRE_SIM

    # --- Determine main simulation count ---
    sim_count = choose_sim_count(prob_p1_pre, prob_p2_pre)

    # --- Run main simulation ---
    outcomes = {p1_name:0, p2_name:0, "Draw":0}
    for _ in range(sim_count):
        winner = simulate_game(players[p1_name], players[p2_name])
        if winner == 1:
            outcomes[p1_name] += 1
        elif winner == 2:
            outcomes[p2_name] += 1
        else:
            outcomes["Draw"] += 1

    # --- Calculate probabilities & SE ---
    prob_p1 = outcomes[p1_name] / sim_count
    prob_p2 = outcomes[p2_name] / sim_count
    prob_draw = outcomes["Draw"] / sim_count

    se_p1 = np.sqrt(prob_p1*(1-prob_p1)/sim_count)
    se_p2 = np.sqrt(prob_p2*(1-prob_p2)/sim_count)
    se_draw = np.sqrt(prob_draw*(1-prob_draw)/sim_count)

    # --- Betting quotes ---
    quote_p1 = 1/prob_p1 if prob_p1>0 else float('inf')
    quote_p2 = 1/prob_p2 if prob_p2>0 else float('inf')
    quote_draw = 1/prob_draw if prob_draw>0 else float('inf')

    # --- Print results ---
    print(f"\n{p1_name} vs {p2_name} ({sim_count} games)")
    print(f"{p1_name}: {prob_p1*100:.1f}% ± {se_p1*100:.1f}% → Quote ≈ {quote_p1:.2f}")
    print(f"{p2_name}: {prob_p2*100:.1f}% ± {se_p2*100:.1f}% → Quote ≈ {quote_p2:.2f}")
    print(f"Draw: {prob_draw*100:.1f}% ± {se_draw*100:.1f}% → Quote ≈ {quote_draw:.2f}")


ModuleNotFoundError: No module named 'board'